# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

In [1]:
!pip install syft

     |████████████████████████████████| 225kB 6.5MB/s 
     |████████████████████████████████| 460kB 40.0MB/s 
     |████████████████████████████████| 204kB 43.2MB/s 
     |████████████████████████████████| 389kB 38.8MB/s 
     |████████████████████████████████| 256kB 39.2MB/s 
     |████████████████████████████████| 81kB 21.2MB/s 
     |████████████████████████████████| 2.1MB 37.8MB/s 
     |████████████████████████████████| 51kB 19.3MB/s 
     |████████████████████████████████| 276kB 39.9MB/s 
     |████████████████████████████████| 122kB 38.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [2]:
import torch as th
from torch import nn, optim
import syft as sy
hook = sy.TorchHook(th)
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")
alice.add_workers([bob, secure_worker])
bob.add_workers([alice, secure_worker])
secure_worker.add_workers([bob, alice])


W0723 15:42:06.240194 140376192493440 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0723 15:42:06.262278 140376192493440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0723 15:42:07.119676 140376192493440 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0723 15:42:07.121312 140376192493440 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0723 15:42:07.122493 140376192493440 base.py:628] Worker alice already exists. Replacing old worker which could 

<VirtualWorker id:secure_worker #objects:0>

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)


In [0]:
alice_data = data[0:2].send(alice)
alice_target = target[0:2].send(alice)

bob_data = data[2:].send(bob)
bob_target = target[2:].send(bob)

In [0]:
#define model
model = nn.Linear(2,1)

In [0]:
for i in range(10):
  alice_model = model.copy().send(alice)
  bob_model = model.copy().send(bob)
  
  alice_opt = optim.SGD(params = alice_model.parameters(), lr=0.1)
  bob_opt = optim.SGD(params= bob_model.parameters(), lr=0.1)
  
  for worker_iter in range(5):
    #Train alice's model
    alice_opt.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_loss =((alice_pred - alice_target)**2).sum()
    alice_loss.backward()
    alice_opt.step()
    alice_loss = alice_loss.get().data
    
    #Train bob's model
    bob_opt.zero_grad()
    bob_pred = bob_model(bob_data)
    bob_loss = ((bob_pred - bob_target)**2).sum()
    bob_loss.backward()
    bob_opt.step()
    bob_loss = bob_loss.get().data
    
  alice_model.move(secure_worker)
  bob_model.move(secure_worker)
  
  with th.no_grad():
    model.weight.set_(((alice_model.weight.data + bob_model.weight.data)/2).get())
    model.bias.set_(((alice_model.bias.data + bob_model.bias.data)/2).get())
    
  print("Alice's loss :" + str(alice_loss) +" Bob's loss:" + str(bob_loss))

Alice's loss :tensor(0.0953) Bob's loss:tensor(0.0669)
Alice's loss :tensor(0.0391) Bob's loss:tensor(0.0269)
Alice's loss :tensor(0.0168) Bob's loss:tensor(0.0107)
Alice's loss :tensor(0.0074) Bob's loss:tensor(0.0041)
Alice's loss :tensor(0.0032) Bob's loss:tensor(0.0015)
Alice's loss :tensor(0.0014) Bob's loss:tensor(0.0005)
Alice's loss :tensor(0.0006) Bob's loss:tensor(0.0002)
Alice's loss :tensor(0.0003) Bob's loss:tensor(3.5616e-05)
Alice's loss :tensor(0.0001) Bob's loss:tensor(5.9529e-06)
Alice's loss :tensor(6.0474e-05) Bob's loss:tensor(2.9651e-06)


In [0]:
pred = model(data)
loss = ((pred - target)**2).sum()
print(pred)
print(loss.data)

tensor([[0.9766],
        [0.9719],
        [0.0208],
        [0.0162]], grad_fn=<AddmmBackward>)
tensor(0.0020)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [0]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [0]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!
import random
Q = 23740629843760239486723

In [0]:
def encrypt(input_no, no_of_shares=3):
  shares = list()
  for i in range(no_of_shares - 1):
    shares.append(random.randint(0, Q))
    
  final_share = Q - (sum(shares)) + input_no
  shares.append(final_share)
  return tuple(shares)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [0]:
def add(a, b):
  c = list()
  assert(len(a) == len(b))
  
  for i in range(len(a)):
    c.append((a[i] + b[i]) % Q)
    
  return tuple(c)

In [0]:
x = encrypt(5, 3)
x

(9253396430582036388340, 6242097992188440955917, 8245135420989762142471)

In [0]:
decrypt(x)

5

In [0]:
y = encrypt(10, 3)
y

(10635101890907921361219, 20466560517945661918827, -7361032565093343793313)

In [0]:
decrypt(y)

10

In [0]:
z = add(x, y)
z

(19888498321489957749559, 2968028666373863388021, 884102855896418349158)

In [0]:
decrypt(z)

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(3.5)

35000

In [0]:
decode(35000)

3.5

In [0]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [0]:
bob._objects

{63630540351: tensor([ 122657067425142073,  705624696924692379, 4235362399363677477,
         3912453810110017651, 4351872078309608219])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:14600044635 -> bob:58983584996]
	-> (Wrapper)>[PointerTensor | me:35378079237 -> alice:9703443659]
	-> (Wrapper)>[PointerTensor | me:15168461196 -> secure_worker:53469013951]
	*crypto provider: me*

In [0]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [0]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
bob._objects

{11852525720: tensor([1451031803950157413,  175055965849360086, 3248471294535871531])}

In [0]:
alice._objects

{19989860470: tensor([  147721880259904049,   -50360961819751422, -2718470698472392728])}

In [0]:
secure_worker._objects

{55699658399: tensor([-1598753684210061362,  -124695004029608464,  -530000596063478503])}

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [3]:
alice.clear_objects()
bob.clear_objects()
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [0]:
compute_nodes = [bob, alice] 

In [0]:
#new_data = data.fix_prec().share(bob, alice, crypto_provider = secure_worker)
#new_data
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [0]:
#new_target= target.fix_prec().share(bob, alice, crypto_provider = secure_worker)
#new_target
alice_data = data[0:2].send(alice)
alice_target = target[0:2].send(alice)

bob_data = data[2:].send(bob)
bob_target = target[2:].send(bob)

In [7]:
alice._objects

{89681886407: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 95384173970: tensor([[1.],
         [1.]], requires_grad=True)}

In [8]:
bob._objects

{19536177582: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 76667740505: tensor([[0.],
         [0.]], requires_grad=True)}

In [0]:
model = nn.Linear(2,1)

In [0]:
'''def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = ((pred.view(-1) - target) ** 2).sum()
    loss.backward()
    loss = loss.get().data
    optimizer.step()
    print("loss" + str(loss))
    return model
  
alice_model = model.copy()
bob_model = model.copy()
  
alice_opt = optim.SGD(alice_model.parameters(), lr=0.01)
bob_opt = optim.SGD(bob_model.parameters(), lr=0.01)

models = [bob_model, alice_model]
params = [list(bob_model.parameters()), list(alice_model.parameters())]
optimizers = [bob_opt, alice_opt]
'''

In [10]:
for i in range(10):
  alice_model = model.copy().send(alice)
  bob_model = model.copy().send(bob)
  
  alice_opt = optim.SGD(alice_model.parameters(), lr=0.01)
  bob_opt = optim.SGD(bob_model.parameters(), lr=0.01)
  
  #params = [list(bob_model.parameters()), list(alice_model.parameters())]
  
  for worker_iter in range(5):
    alice_opt.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_loss = ((alice_pred - alice_target) ** 2).sum()
    alice_loss.backward()
    alice_opt.step()
    alice_loss = alice_loss.get().data
    
    bob_opt.zero_grad()
    bob_pred = bob_model(bob_data)
    bob_loss = ((bob_pred - bob_target) ** 2).sum()
    bob_loss.backward()
    bob_opt.step()
    bob_loss = bob_loss.get().data
  
  '''for remote_index in range(len(compute_nodes)):
    
    if(remote_index == 0):
      data, target = bob_data, bob_target
    else:
      data, target = alice_data, alice_target
      
    models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])
    
  new_params = list()
  for param_i in range(len(params[0])):
    worker_param = list()
    for index in range(len(compute_nodes)):
      encrypted_param = params[index][param_i].copy().fix_prec().share(bob, alice, crypto_provider = secure_worker)
      worker_param.append(encrypted_param.get())
      
    new_param = (worker_param[0] + worker_param[1]).get().float_prec()/2
    new_params.append(new_param)'''
  
  alice_wt = alice_model.weight.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get()
  bob_wt = bob_model.weight.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get()
  alice_bias = alice_model.bias.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get() 
  bob_bias = bob_model.bias.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get()
  
  with th.no_grad():
    model.weight.set_((( alice_wt + bob_wt )/2).get().float_prec())
    model.bias.set_((( alice_bias + bob_bias)/2).get().float_prec())
    
    '''for index in range(len(compute_nodes)):
      for param_i in range(len(params[index])):
        params[index][param_i].set_(new_params[param_i])'''
    
  print("Alice's loss:" + str(alice_loss) + "  Bob's loss:" + str(bob_loss))

AttributeError: ignored

In [11]:
bob._objects

{19536177582: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 29174673645: tensor([[-0.2226],
         [-0.2091]], grad_fn=<AddmmBackward>), 76667740505: tensor([[0.],
         [0.]], requires_grad=True), 83026907985: Parameter containing:
 tensor([[-0.0091,  0.6633]], requires_grad=True), 90864877396: Parameter containing:
 tensor([-0.2004], requires_grad=True)}

In [12]:
alice._objects

{46578704512: tensor([[0.5915],
         [0.5830]], grad_fn=<AddmmBackward>),
 70891811136: tensor([[0.0166, 0.7639]]),
 73359939689: Parameter containing:
 tensor([[0.0166, 0.7639]], requires_grad=True),
 87506759407: Parameter containing:
 tensor([-0.1479], requires_grad=True),
 89681886407: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 95384173970: tensor([[1.],
         [1.]], requires_grad=True)}